# LoL Stats Visualization

In [1]:
import backend.downloader as dl
import backend.assets as assets
from backend.db import Database
import pandas as pd
import altair as alt

## Update all assets

In [2]:
assets.update_all()

Updating assets using constants.py urls...
Wrote 153 lines to 'champions table in DB
Updated champions asset (json & db table).
Wrote 82 lines to 'queues table in DB
Updated queues asset (json & db table).
Wrote 15 lines to 'maps table in DB
Updated maps asset (json & db table).
Wrote 14 lines to 'summoner_spells table in DB
Updated summoner_spells asset (json & db table).
All assets updated


## Download data

In [17]:
SUMMONERS = "Helikk" #["Helikk", "marcosalome", "smallfatant", "Vempramim", "Raketes", "Raketex", "MadameTripas"]
dl.get_data(SUMMONERS)

Downloaded 1 Summoner profile(s).
Downloaded 811 match ids for 'Helikk'.
Total downloaded match ids: 811.
Read 810 lines from 'match_details' table in DB.
Compared DB with downloaded data: 1 new matches found.
Getting data on 1 matches.
Finished processing 1 matches.
Data on 1 matches downloaded.


In [18]:
with Database("./backend/db/data_normalized.db") as db:
    md = db.read_full_table("match_details")
    t = db.read_full_table("teams")
    p = db.read_full_table("participants")
    pi = db.read_full_table("participant_identities")

Read 811 lines from 'match_details' table in DB.
Read 1622 lines from 'teams' table in DB.
Read 8110 lines from 'participants' table in DB.
Read 8110 lines from 'participant_identities' table in DB.


In [19]:
with Database("./backend/db/assets.db") as db:
    c = db.read_full_table("champions")
    q = db.read_full_table("queues")

Read 152 lines from 'champions' table in DB.
Read 82 lines from 'queues' table in DB.


In [20]:
df = pd.merge(p, c, on=["championId"])
df = pd.merge(df, md, on=["gameId"])
df = pd.merge(df, q, on=["queueId"])
df = pd.merge(df, pi, on=["gameId", "participantId"])
df.columns = df.columns.str.replace(".", "_")

In [21]:
df["datetime"] = pd.to_datetime(df["gameCreation"], unit="ms")


In [22]:
df.columns[100:200]
len(df)

8110

In [23]:
gb_scd = df[["player_summonerName", "champion", "datetime"]].groupby(by=["player_summonerName", "champion"]).max().reset_index().rename(columns={"datetime": "last_played_datetime"})
df = pd.merge(df, gb_scd, on=["player_summonerName", "champion"])
df["last_game"] = df["datetime"] == df["last_played_datetime"]


In [24]:
df.to_csv("./backend/outputs/denormalized_data.csv")

In [25]:
df["year"] = pd.DatetimeIndex(df["datetime"]).year
df.dtypes

participantId                    int64
teamId                           int64
championId                       int64
spell1Id                         int64
spell2Id                         int64
                             ...      
player_profileIcon               int64
datetime                datetime64[ns]
last_played_datetime    datetime64[ns]
last_game                         bool
year                             int64
Length: 166, dtype: object

In [27]:

data = df.loc[(df["map"] == "Summoner's Rift") & (df["year"] >= 2020) & (df["queue"].isin(["5v5 Blind Pick games", "5v5 Draft Pick games", "5v5 Ranked Flex games", "5v5 Ranked Solo games	"]))]

In [90]:
data.loc[data["gameDuration"] >= 12*60][["gameId", "participantId", "teamId", "champion", "stats_neutralMinionsKilled", "stats_totalMinionsKilled", "datetime"]].sort_values(by=["datetime", "gameId", "participantId"], ascending=False)[0:20]

,gameId,participantId,teamId,champion,stats_neutralMinionsKilled,stats_totalMinionsKilled,datetime
1989,4969134469,10,200,Malphite,4,40,2020-12-08 15:28:45.967
1986,4969134469,9,200,Yasuo,12,172,2020-12-08 15:28:45.967
362,4969134469,8,200,Jarvan IV,134,34,2020-12-08 15:28:45.967
1994,4969134469,7,200,Camille,15,189,2020-12-08 15:28:45.967
1997,4969134469,6,200,Ziggs,0,121,2020-12-08 15:28:45.967
1985,4969134469,5,100,Kayn,128,51,2020-12-08 15:28:45.967
1988,4969134469,4,100,Jhin,15,140,2020-12-08 15:28:45.967
1995,4969134469,3,100,Brand,1,46,2020-12-08 15:28:45.967
1998,4969134469,2,100,Gnar,0,203,2020-12-08 15:28:45.967
1996,4969134469,1,100,Jax,12,143,2020-12-08 15:28:45.967


In [95]:
list(set(data.loc[data["gameDuration"] >= 12*60].sort_values(by=["datetime"], ascending=False)["gameId"][0:100]))

[4964963104,
 4968423394,
 4964888834,
 4966840804,
 4969134469,
 4966763109,
 4964797716,
 4968327414,
 4968159452,
 4969131679]

In [120]:
list(set(data.loc[data["gameDuration"] >= 12*60]["gameId"]))[0]

4792395776

In [124]:
from riotwatcher import LolWatcher
import roleml
from backend.utils import retry_function

watcher = LolWatcher("RGAPI-e36cb6c9-8d3b-4f82-aef4-d9e86dd18444")
predictions = {}

for gameId in list(set(data.loc[data["gameDuration"] >= 12*60]["gameId"])):
    timeline = watcher.match.timeline_by_match(region="euw1", match_id=gameId)
    match = watcher.match.by_id(region="euw1", match_id=gameId)
    predictions[gameId] = roleml.predict(match, timeline)
print(predictions)


 4: 'jungle', 5: 'supp', 6: 'bot', 7: 'jungle', 8: 'top', 9: 'mid', 10: 'supp'}, 4752149535: {1: 'jungle', 2: 'supp', 3: 'top', 4: 'bot', 5: 'mid', 6: 'supp', 7: 'top', 8: 'mid', 9: 'bot', 10: 'jungle'}, 4655901727: {1: 'jungle', 2: 'bot', 3: 'mid', 4: 'top', 5: 'supp', 6: 'jungle', 7: 'mid', 8: 'supp', 9: 'bot', 10: 'top'}, 4633678882: {1: 'top', 2: 'jungle', 3: 'mid', 4: 'bot', 5: 'supp', 6: 'supp', 7: 'bot', 8: 'jungle', 9: 'top', 10: 'mid'}, 4618626087: {1: 'jungle', 2: 'mid', 3: 'top', 4: 'bot', 5: 'supp', 6: 'jungle', 7: 'supp', 8: 'bot', 9: 'mid', 10: 'top'}, 4681589803: {1: 'bot', 2: 'supp', 3: 'mid', 4: 'top', 5: 'jungle', 6: 'jungle', 7: 'mid', 8: 'supp', 9: 'bot', 10: 'top'}, 4682439734: {1: 'jungle', 2: 'top', 3: 'mid', 4: 'supp', 5: 'bot', 6: 'top', 7: 'mid', 8: 'jungle', 9: 'supp', 10: 'bot'}, 4928281655: {1: 'jungle', 2: 'top', 3: 'mid', 4: 'supp', 5: 'bot', 6: 'mid', 7: 'bot', 8: 'top', 9: 'supp', 10: 'jungle'}, 4913925185: {1: 'jungle', 2: 'supp', 3: 'bot', 4: 'mid', 5

In [126]:
pred = pd.DataFrame.from_dict(predictions, orient="index").reset_index().melt(id_vars="index", var_name="participantId", value_name="position").rename(columns={"index": "gameId"})
rel_cols = ["gameId", "champion", "player_summonerName", "stats_win", "gameDuration_mins", "stats_visionScore", "dtc_pm", "ge_pm", "cs_pm", "datetime", "participantId", "position"]
data = pd.merge(data, pred, on=["gameId", "participantId"])[rel_cols].sort_values(by=["datetime", "participantId"])


In [141]:
data.to_csv("./backend/outputs/position_data.csv")

In [140]:
gb3 = data.loc[data["player_summonerName"] == "Helikk"].groupby(by=["position"]).aggregate({"gameId": "count", "stats_win": "sum", "gameDuration_mins": "mean", "stats_visionScore": "mean", "dtc_pm": "mean", "ge_pm": "mean", "cs_pm": "mean"}).sort_values(by=["position"], ascending=True).reset_index()

gb3["win_pct"] = gb3["stats_win"] / gb3["gameId"]
gb3

,position,gameId,stats_win,gameDuration_mins,stats_visionScore,dtc_pm,ge_pm,cs_pm,win_pct
0,bot,89,46,30.66,11.85,502.08,361.70,5.15,0.52
1,jungle,88,44,30.13,20.58,348.73,339.46,4.46,0.50
2,mid,70,32,31.06,14.96,521.57,346.28,4.82,0.46
3,supp,46,28,29.51,33.80,360.90,279.29,0.88,0.61
4,top,52,22,31.22,14.81,508.21,337.82,5.16,0.42


In [139]:
gb2 = data.loc[data["player_summonerName"] == "Helikk"].groupby(by=["champion", "position"]).aggregate({"gameId": "count", "stats_win": "sum", 
                                                                                                                            "gameDuration_mins": "mean", 
                                                                                                                            "stats_visionScore": "mean", 
                                                                                                                            "dtc_pm": "mean", 
                                                                                                                            "ge_pm": "mean", "cs_pm": "mean"}).sort_values(by=["champion", "position"], ascending=True).reset_index()

gb2["win_pct"] = gb2["stats_win"] / gb2["gameId"]
gb2.loc[gb2["gameId"] >= 2]

,champion,position,gameId,stats_win,gameDuration_mins,stats_visionScore,dtc_pm,ge_pm,cs_pm,win_pct
0,Ahri,bot,2,1,27.55,14.50,413.49,326.66,4.33,0.50
1,Ahri,mid,9,1,32.44,15.89,614.57,329.97,4.64,0.11
2,Akali,mid,2,1,27.05,8.00,358.78,324.37,4.64,0.50
3,Akali,top,3,0,26.23,11.33,457.03,306.86,4.36,0.00
4,Alistar,supp,3,2,32.22,30.67,290.14,271.07,0.93,0.67
5,Amumu,jungle,5,3,23.42,17.20,295.57,351.74,5.31,0.60
7,Bard,supp,2,1,28.02,31.50,254.37,256.13,1.25,0.50
8,Blitzcrank,supp,5,2,29.53,31.00,333.00,277.93,1.12,0.40
10,Caitlyn,bot,6,1,31.89,12.17,486.37,344.43,5.13,0.17
11,Camille,top,2,0,31.36,12.50,447.08,357.72,4.92,0.00


In [53]:
data["gameDuration_mins"] = data["gameDuration"] / 60
data["cs_pm"] = (data["stats_neutralMinionsKilled"] + data["stats_totalMinionsKilled"]) / (data["gameDuration_mins"])
data["ge_pm"] = (data["stats_goldEarned"]) / (data["gameDuration_mins"])
data["dtc_pm"] = (data["stats_totalDamageDealtToChampions"]) / (data["gameDuration_mins"])
rel_cols = ["gameId", "champion", "player_summonerName", "stats_win", "gameDuration_mins", "stats_visionScore", "dtc_pm", "ge_pm", "cs_pm"]

pd.options.display.float_format = '{:,.2f}'.format
gb = data.loc[data["player_summonerName"] == "Helikk"][rel_cols].groupby(by=["player_summonerName", "champion"]).aggregate({"gameId": "count", 
                                                                                                                            "stats_win": "sum", 
                                                                                                                            "gameDuration_mins": "mean", 
                                                                                                                            "stats_visionScore": "mean", 
                                                                                                                            "dtc_pm": "mean", 
                                                                                                                            "ge_pm": "mean", 
                                                                                                                            "cs_pm": "mean"}).sort_values(by="gameId", ascending=False).reset_index()
gb["win_pct"] = gb["stats_win"] / gb["gameId"]
gb.loc[gb["gameId"] >= 4]

,player_summonerName,champion,gameId,stats_win,gameDuration_mins,stats_visionScore,dtc_pm,ge_pm,cs_pm,win_pct
0,Helikk,Miss Fortune,25,15,28.79,10.96,514.25,391.29,5.67,0.60
1,Helikk,Quinn,15,6,28.51,13.33,494.71,341.06,5.17,0.40
2,Helikk,Ezreal,15,8,30.28,8.40,440.17,326.44,4.72,0.53
3,Helikk,Syndra,13,6,29.40,12.77,481.33,339.39,4.90,0.46
4,Helikk,Neeko,13,7,37.23,25.62,591.71,360.47,5.15,0.54
5,Helikk,Diana,12,5,29.26,13.67,537.35,360.84,5.19,0.42
6,Helikk,Kha'Zix,12,6,23.66,14.58,341.13,340.23,4.19,0.50
7,Helikk,Lucian,11,5,28.93,14.09,522.32,374.89,5.59,0.45
8,Helikk,Ahri,11,2,31.55,15.64,578.01,329.37,4.58,0.18
9,Helikk,Lux,10,6,29.91,35.20,528.88,293.08,1.02,0.60


In [31]:
alt.Chart(data.loc[data["player_summonerName"] == "Helikk"]).mark_bar().encode(
    alt.Y("champion:N", sort="-x"),
    x="count(gameId):Q",
    tooltip="count(gameId):Q"
)

alt.Chart(...)

## Get data from DBs

In [ ]:
# Read from DB to dataframe
NORMALIZED_DB_PATH = "./backend/db/data_normalized.db"

nconn = db.create_connection(NORMALIZED_DB_PATH)
md = db.read_full_table(nconn, "match_details")
t = db.read_full_table(nconn, "teams")
p = db.read_full_table(nconn, "participants")
pi = db.read_full_table(nconn, "participant_identities")
db.close_connection(nconn)

In [ ]:
# Add queue and map info
ASSETS_PATH = "./backend/db/assets.db"

aconn = db.create_connection(ASSETS_PATH)
qs = db.read_full_table(aconn, "queues")
db.close_connection(aconn)

## Match details exploration
### Create additional variables 

In [ ]:
# Get date from gameCreation
md["datetime"] = pd.to_datetime(md["gameCreation"], unit="ms")

# Get approximate match duration
md["gameDurationMinutes"] = round(md["gameDuration"] / 60, 0)
md["gameDurationMinutes"] = md["gameDurationMinutes"].astype("int16")

# Add queue names - NEXUSBLITZ games may be lost as their queue seems to not exists in the queues dataset
if not "queue" in md.columns:
    md = pd.merge(md, qs, on="queueId")

md

### Basic count charts

#### gameMode

In [ ]:
common_args = [
    alt.X("count():Q", axis=alt.Axis(grid=True, ticks=True)),
]

mode_text = alt.Chart().mark_text(dx=10).encode(
    *common_args,
    alt.Y("gameMode:N"),
    alt.Text("count():Q"),
)

mode_counts = alt.Chart().mark_bar().encode(
    *common_args,
    alt.Y("gameMode:N"),
    alt.Color("datetime:N", timeUnit="year"),
    alt.Tooltip("count():Q")
)

alt.layer(mode_counts, mode_text, data=md)

In [ ]:
alt.layer(mode_counts, mode_text, data=md).facet(
    row=alt.Row("datetime:N", timeUnit="yearmonth")
)

In [ ]:
alt.Chart(md).mark_tick(width=40, thickness=3).encode(
    alt.X("gameMode:N"),
    alt.Y("gameDurationMinutes:Q"),
    alt.Color("count():Q"),
    tooltip=[alt.Tooltip("count():Q"), alt.Tooltip("gameDurationMinutes:Q")]
).configure_view(width=350, height=350)

In [ ]:
alt.Chart(md).transform_density(
    'gameDurationMinutes',
    as_=['gameDurationMinutes', 'density'],
    groupby=['gameMode']
).mark_area(orient='horizontal').encode(
    y='gameDurationMinutes:Q',
    color='gameMode:N',
    x=alt.X(
        'density:Q',
        stack='center',
        impute=None,
        title=None,
        axis=alt.Axis(labels=False, values=[0],grid=False, ticks=True),
    ),
    column=alt.Column(
        'gameMode:N',
        header=alt.Header(
            titleOrient='bottom',
            labelOrient='bottom',
            labelPadding=0,
        ),
    )
).configure_facet(
    spacing=0
)

#### gameDurationMinutes

In [ ]:
duration_text = alt.Chart().mark_text(dx=10).encode(
    *common_args,
    alt.Y("gameDurationMinutes:N"),
    alt.Text("count():Q"),
)

duration_counts = alt.Chart().mark_bar().encode(
    *common_args,
    alt.Y("gameDurationMinutes:N"),
    alt.Color("datetime:N", timeUnit="year"),
    alt.Tooltip("count():Q")
)

alt.layer(duration_counts, duration_text, data=md)

In [ ]:
duration_counts = duration_counts.encode(
    alt.Color("gameMode:N")
)

alt.layer(duration_counts, duration_text, data=md)

#### datetime

In [ ]:
def datetime_layer_chart(time_unit):
    datetime_text = alt.Chart().mark_text(dx=10).encode(
        *common_args,
        alt.Text("count():Q"),
        alt.Y("datetime:N", timeUnit=time_unit)
    )

    datetime_counts = alt.Chart().mark_bar().encode(
        *common_args,
        alt.Y("datetime:N", timeUnit=time_unit)
    )

    return alt.layer(datetime_counts, datetime_text, data=md)

datetime_layer_chart("year")

In [ ]:
datetime_layer_chart("yearmonth")

In [ ]:
datetime_layer_chart("day")

In [ ]:
datetime_layer_chart("hours")

In [ ]:
alt.Chart(md).mark_circle().encode(
    alt.X("datetime:T", timeUnit="hoursminutes"),
    alt.Y("gameDurationMinutes:Q"),
    #alt.Color("datetime:N", timeUnit="day"),
    #alt.Size("count()"),
    tooltip=[alt.Tooltip("datetime:T", timeUnit="hoursminutes"),
        alt.Tooltip("gameDurationMinutes:Q")],
)

In [ ]:
hm = alt.Chart(md).mark_rect().encode(
    alt.X("datetime:O", timeUnit="day"),
    alt.Y("gameMode:O"),
    alt.Color("count()")
)

text = alt.Chart(md).mark_text().encode(
    alt.X("datetime:O", timeUnit="day"),
    alt.Y("gameMode:O"),
    alt.Text("count()")
)

(hm + text).configure_view(width=400, height=200)

### Maps

In [ ]:
alt.Chart(md).mark_bar().encode(
    alt.X("map:N"),
    alt.Y("count():Q")
).configure_view(width=200)

In [ ]:
lines = alt.Chart(md).mark_line(point=True).encode(
    alt.X("datetime:T", timeUnit="yearmonth"),
    alt.Y("count():Q"),
    alt.Color("map:N"),
    alt.Tooltip("count():Q")
) 
bars = alt.Chart(md).transform_calculate(diff="datum.map == 'Howling Abyss' ? 1 : -1").mark_bar().encode(
    alt.X("datetime:T", timeUnit="yearmonth"),
    alt.Y("sum(diff):Q"),
    alt.Tooltip("sum(diff):Q")
)

lines & bars

#### Queues

In [ ]:
alt.Chart(md).mark_circle(size=50).encode(
    alt.Y("queue:N"),
    alt.X("gameDurationMinutes:Q"),
    alt.Color("count():Q"),
    alt.Tooltip("count():Q")
).configure_view(height=300, width=500)

In [ ]:
squares = alt.Chart(md).mark_rect().encode(
    alt.X("datetime:N", timeUnit="day"),
    alt.Y("queue:N"),
    alt.Color("count():Q"),
    alt.Tooltip("count():Q"),
)

text = alt.Chart(md).mark_text().encode(
    alt.X("datetime:N", timeUnit="day"),
    alt.Y("queue:N"),
    alt.Text("count():Q")
)

(squares + text).configure_view(height=300, width=350)

## Participants exploration

In [ ]:
TBC